In [1]:
import pickle, torch
from torch import nn
from torch.utils.data import DataLoader
from torchsummary import summary
import time

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
# load dataset from .pickle file generated by pickle_dataset
dataset = {}
with open('dataset.pickle', 'rb') as file:
    dataset = pickle.load(file)

In [4]:
class MathClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(1, 10, 6),
            nn.ReLU(),
            nn.AvgPool2d(2),
            nn.Conv2d(10, 20, 6),
            nn.ReLU(),
            nn.AvgPool2d(3),
            nn.Flatten(),
            nn.Linear(500, 82),
        )
    
    def forward(self, x):
        return self.layers.forward(x)

summary(MathClassifier().to(device), (1, 45, 45))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 40, 40]             370
              ReLU-2           [-1, 10, 40, 40]               0
         AvgPool2d-3           [-1, 10, 20, 20]               0
            Conv2d-4           [-1, 20, 15, 15]           7,220
              ReLU-5           [-1, 20, 15, 15]               0
         AvgPool2d-6             [-1, 20, 5, 5]               0
           Flatten-7                  [-1, 500]               0
            Linear-8                   [-1, 82]          41,082
Total params: 48,672
Trainable params: 48,672
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 0.35
Params size (MB): 0.19
Estimated Total Size (MB): 0.54
----------------------------------------------------------------


In [16]:
# Hyper Parameters
LEARNING_RATE = 0.001
BATCH_SIZE = 16
EPOCHS = 5

model = MathClassifier().to(device) # reset model
train_dataloader = DataLoader(dataset['train'], BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(dataset['val'], BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(dataset['test'], BATCH_SIZE, shuffle=True)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)

In [17]:
def train_loop():
    # statistics
    num_correct = 0
    num_total = 0
    for X, y in iter(train_dataloader):
        # forward pass
        logits = model.forward(X.to(device))

        # calculate gradient
        loss = criterion(logits, y.to(device))
        loss.backward()

        # backward pass
        optimizer.step()
        optimizer.zero_grad()

        # update statistics
        pred_y = torch.argmax(logits, 1)
        for i in range(len(pred_y)):
            num_correct += pred_y[i] == y[i]
            num_total += 1
    
    accuracy = 100*(num_correct/num_total)
    print(f'  Training Accuracy: {accuracy:.2f}%')

def val_loop():
    # statistics
    num_correct = 0
    num_total = 0
    with torch.no_grad():
        for X, y in iter(val_dataloader):
            # forward pass
            logits = model.forward(X.to(device))

            # update statistics
            pred_y = torch.argmax(logits, 1)
            for i in range(len(pred_y)):
                num_correct += pred_y[i] == y[i]
                num_total += 1
    
    accuracy = 100*(num_correct/num_total)
    print(f'  Validation Accuracy: {accuracy:.2f}%')

start_time = time.time()
for epoch in range(EPOCHS):
    # start epoch
    print(f'Epoch [{epoch + 1}/{EPOCHS}]')

    # run train and val loop
    train_loop()
    val_loop()

    # print time
    t = time.time() - start_time
    print(f'  Elapsed time: {int(t/60)}m {int(t%60)}s')

Epoch [1/5]
  Training Accuracy: 76.72%
  Validation Accuracy: 84.59%
  Elapsed time: 2m 20s
Epoch [2/5]
  Training Accuracy: 88.02%
  Validation Accuracy: 89.45%
  Elapsed time: 4m 47s
Epoch [3/5]
  Training Accuracy: 90.24%
  Validation Accuracy: 90.48%
  Elapsed time: 6m 50s
Epoch [4/5]
  Training Accuracy: 91.30%
  Validation Accuracy: 91.56%
  Elapsed time: 9m 9s
Epoch [5/5]
  Training Accuracy: 92.01%
  Validation Accuracy: 92.11%
  Elapsed time: 11m 47s


In [ ]:
def test_loop():
    # statistics
    num_correct = 0
    num_total = 0
    with torch.no_grad():
        for X, y in iter(test_dataloader):
            # forward pass
            logits = model.forward(X.to(device))

            # update statistics
            pred_y = torch.argmax(logits, 1)
            for i in range(len(pred_y)):
                num_correct += pred_y[i] == y[i]
                num_total += 1
    
    accuracy = 100*(num_correct/num_total)
    print(f'Accuracy: {accuracy:.2f}%')
test_loop()